In [1]:
# TEAM UNI:  Sagat Damani ssd2144 Eric Jhonson  efj2106   Aman Shankar as5171
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [3]:
# Load the data
data_root_path = './cifar10-hw1/'
X_train_orig, y_train_orig = get_train_data(data_root_path) # this may take a few minutes
X_test_orig = get_images(data_root_path + 'test')
print('Data loading done')

{'horse': 7, 'automobile': 1, 'deer': 4, 'dog': 5, 'frog': 6, 'cat': 3, 'truck': 9, 'ship': 8, 'airplane': 0, 'bird': 2}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [4]:
#Perform train-validation split (90-10)
X_train, y_train = X_train_orig, y_train_orig
indices = np.random.permutation(X_train.shape[1])
split_point = int(len(indices)* 0.9)
print(split_point)
training_indices, validation_indices = indices[:split_point], indices[split_point:]

X_train, X_validation = X_train[: , training_indices], X_train[:, validation_indices]
y_train, y_validation = y_train[training_indices], y_train[validation_indices]
print(X_train.shape)
print(X_validation.shape)

45000
(3072, 45000)
(3072, 5000)


In [5]:
X_train = np.transpose(X_train).astype("float32")
y_train = y_train.astype("float32")
print(X_train.shape)
print(y_train.shape)

X_validation = np.transpose(X_validation).astype("float32")
y_validation = y_validation.astype("float32")


(45000, 3072)
(45000,)


In [11]:
def cnn_model_fmine(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  tf.summary.histogram("conv_layer1",conv1)
  conv1 = tf.layers.batch_normalization(conv1)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  tf.summary.histogram("conv_layer2",conv2)
    
  conv2 = tf.layers.batch_normalization(conv2)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  print(conv2.get_shape().as_list())
  print(pool2.get_shape().as_list())

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  tf.summary.histogram("Last_dense_layer",dense)

  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=tf.nn.softmax(logits, name="softmax_tensor"))

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)# Train the model

In [12]:
# Train the model
print(X_train.shape)
print(y_train.shape)

my_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fmine, model_dir="/tmp/1_temp", config=tf.contrib.learn.RunConfig(save_summary_steps=100))
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
my_classifier.train(
    input_fn=train_input_fn,
    steps=40000)

(45000, 3072)
(45000,)
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0c75beb190>, '_model_dir': '/tmp/1_temp', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
[100, 16, 16, 64]
[100, 8, 8, 64]
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/1_temp/model.ckpt.
INFO:tensorflow:loss = 2.34155, step = 1
INFO:tensorflow:global_step/sec: 47.7357
INFO:tensorflow:loss = 2.2371, step = 101 (2.096 sec)
INFO:tensorflow:global_step/sec: 50.6744
INFO:tensorflow:

In [8]:
# Evaluate the model and print results (training accuracy)
# summary_hook = tf.train.SummarySaverHook(
#     SAVE_EVERY_N_STEPS,
#     output_dir='/tmp/tf',
#     summary_op=tf.summary.merge_all())
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_results = my_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

[None, 16, 16, 64]
[None, 8, 8, 64]
INFO:tensorflow:Starting evaluation at 2017-10-30-22:04:31
INFO:tensorflow:Restoring parameters from /tmp/1/model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2017-10-30-22:04:34
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.999556, global_step = 40000, loss = 0.00662368
{'loss': 0.006623676, 'global_step': 40000, 'accuracy': 0.99955553}


In [9]:
# Evaluate the model and print results (validation accuracy)
print(X_validation.shape)
print(y_validation.shape)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_validation},
    y=y_validation,
    num_epochs=1,
    shuffle=False)
eval_results = my_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

(5000, 3072)
(5000,)
[None, 16, 16, 64]
[None, 8, 8, 64]
INFO:tensorflow:Starting evaluation at 2017-10-30-22:04:36
INFO:tensorflow:Restoring parameters from /tmp/1/model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2017-10-30-22:04:36
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.7274, global_step = 40000, loss = 1.28079
{'loss': 1.2807904, 'global_step': 40000, 'accuracy': 0.7274}


In [45]:
#Run model on test data and output .npy corresponding to last layer
test_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fmine, model_dir="/tmp/1")
X_test = np.transpose(X_test_orig).astype("float32")
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    num_epochs=1,
    shuffle=False)
predict_results = list(test_classifier.predict(input_fn=eval_input_fn))
predict_results = np.transpose(np.array(predict_results))
print(predict_results.shape)
np.save('cnn_result', predict_results)
print(predict_results[:][:10])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/1', '_save_summary_steps': 100}
[None, 16, 16, 64]
[None, 8, 8, 64]
INFO:tensorflow:Restoring parameters from /tmp/1/model.ckpt-40000
(10, 10000)
[[  3.67666502e-03   4.76384548e-06   2.41645798e-01 ...,   1.93987987e-07
    5.86509287e-01   1.43278955e-09]
 [  2.22291583e-05   4.56307828e-03   2.10764171e-07 ...,   2.24644413e-13
    4.04519469e-01   2.07254902e-09]
 [  2.41099242e-05   2.65492736e-14   4.63216096e-01 ...,   6.82551637e-02
    3.12236580e-03   1.20018249e-05]
 ..., 
 [  1.40715861e-06   5.11823794e-18   3.22219705e-07 ...,   7.05243656e-06
    1.75660516e-05   9.57332313e-01]
 [  7.68048039e-07   9.95432138e-01   6.13669166e-03 ...,   1.34608172e-07
    1.62900978e-05   1.717